In [1]:
# Recurrent neural network (RNN)
%matplotlib inline

import tensorflow as tf
from tensorflow.contrib import rnn


In [2]:
class SeriesPredictor:
    def __init__(self, input_dim, seq_size, hidden_dim=10):
        tf.reset_default_graph()
        self.input_dim = input_dim
        self.seq_size = seq_size
        self.hidden_dim = hidden_dim
        
        self.W_out = tf.Variable(tf.random.normal(shape=(self.hidden_dim, 1)), name='W_out')
        self.b_out = tf.Variable(tf.random.normal([1]), name='b_out')
        
        self.x = tf.placeholder(tf.float32, shape=[None, seq_size, input_dim])
        self.y = tf.placeholder(tf.float32, shape=[None, seq_size])
        
        self.cost = tf.reduce_mean(tf.square(self.y-self.model()))
        self.train_op = tf.train.AdamOptimizer().minimize(self.cost)
        
        self.saver = tf.train.Saver()
        
    def model(self):
        cell = rnn.BasicLSTMCell(self.hidden_dim, reuse=tf.AUTO_REUSE)
        output, states = tf.nn.dynamic_rnn(cell, self.x, dtype=tf.float32)
        num_examples = tf.shape(self.x)[0]
        W_repeated = tf.tile(tf.expand_dims(self.W_out, 0), [num_examples, 1, 1])
        out = tf.matmul(output, W_repeated)+self.b_out
        return tf.squeeze(out)
    
    def train(self, train_x, train_y):
        with tf.Session() as sess:
            tf.get_variable_scope().reuse_variables()
            sess.run(tf.global_variables_initializer())
            for i in range(1000):
                _, mse = sess.run([self.train_op, self.cost], feed_dict={self.x: train_x, self.y: train_y})
                if i % 100 == 0:
                    print(i, mse)
            save_path = self.saver.save(sess, './models/rnn_model.ckpt')
            print('Model saved to {}'.format(save_path))
            
    def test(self, test_x):
        with tf.Session() as sess:
            tf.get_variable_scope().reuse_variables()
            self.saver.restore(sess, './models/rnn_model.ckpt')
            output = sess.run(self.model(), feed_dict= {self.x: test_x})
            print(output)


In [4]:
if __name__ == '__main__':
    predictor = SeriesPredictor(input_dim=1, seq_size= 4, hidden_dim= 10)
    train_x = [[[1], [2], [5], [6]],
               [[5], [7], [7], [8]],
               [[3], [4], [5], [7]]]
    train_y = [[1, 3, 7, 11],
               [5, 12, 14, 15],
               [3, 7, 9, 12]]
    predictor.train(train_x, train_y)
    test_x = [[[1], [2], [3], [4]],
              [[4], [5], [6], [7]]]
    predictor.test(test_x)





0 127.85376
100 72.83297


200 33.022873
300 16.28807


400 5.947744
500 3.9981902


600 3.3360784
700 2.7956407


800 2.2188892
900 1.7084445


Model saved to ./models/rnn_model.ckpt
INFO:tensorflow:Restoring parameters from ./models/rnn_model.ckpt


[[ 0.61828494  2.9523857   5.7788334   7.7803984 ]
 [ 3.8665962   8.957546   11.408652   12.224773  ]]
